In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
from multiprocessing import Pool
import time,gc

In [2]:
# f21:上一天非周期用户的数量(一天只有两次刷卡)
# 分时刻重复上述特征

In [3]:
# 统计进出站每天的变化情况
def get_daily_activity(i):
    if i<10:
        s = '0'+str(i)
    else:
        s = str(i)
    day_activ = pd.read_csv('data/Metro_train/record_2019-01-%s.csv'%s)
    day_activ['time'] = pd.to_datetime(day_activ.time)
    day_activ['stationID'] = day_activ['stationID']
    day_activ['deviceID'] = day_activ['deviceID']
    day_activ['status'] = day_activ['status']
    day_activ['payType'] = day_activ['payType']
    day_activ['lineID'] = day_activ['lineID'].apply(lambda x: {'A':0,'B':2,'C':1}[x])
    return day_activ

def construct_df():
    final = pd.DataFrame()
    for i in range(81):
        temp = pd.DataFrame()
        temp['minute'] = list(range(1440))
        temp['stationID'] = i
        final = final.append(temp)
    final['minute'] = final['minute']/10
    return final


In [4]:
all_df = pd.DataFrame()

#for date in range(7,26):
for date in [12,13,14,19,20,21,26,28]:
    final = construct_df()
    day_activ = get_daily_activity(date)
    day_activ = day_activ[day_activ.status==1]
    userid_count = day_activ.groupby(['userID'],as_index=False).agg({'status':'count'})\
    .rename(columns={'status':'come_count'})
    user_id_group = userid_count[userid_count.come_count==1]['userID']
    user_id_group = set(user_id_group.values)
    day_activ['once_come'] = day_activ['userID'].apply(lambda x:1 if x in user_id_group else 0)
    user_id_group = userid_count[userid_count.come_count==2]['userID']
    user_id_group = set(user_id_group.values)
    day_activ['twice_come'] = day_activ['userID'].apply(lambda x:1 if x in user_id_group else 0)

    df = pd.DataFrame()
    for i in range(10):
        day_activ['minute'] = (day_activ['time'].dt.minute-i+day_activ['time'].dt.hour*60)//10+0.1*i
        day_activ['minute'] = day_activ['minute'].apply(lambda x:143+1+x if x<0 else x)
        df = df.append(day_activ)
    df.sort_values(by='minute',inplace=True)
    come = df
    # 上一天非周期用户的数量(一天只有两次刷卡) 进站一次 进站二次 比率
    f1 = come.groupby(['stationID','minute'],as_index=False).agg({'once_come':'sum'}).\
    rename(columns={'once_come':'once_come_by_s_m'})
    final = final.merge(f1,on=['stationID','minute'],how='left')
    f2 = come.groupby(['stationID','minute'],as_index=False).agg({'twice_come':'sum'}).rename(columns={'twice_come':'twice_come_by_s_m'})
    final = final.merge(f2,on=['stationID','minute'],how='left')
    final['twice_to_once_come_s_m'] = final['once_come_by_s_m']/final['twice_come_by_s_m']
    
    f3 = come.groupby('stationID').agg({'once_come':'sum'}).rename(columns={'once_come':'once_come_by_s'})
    final = final.merge(f3,on='stationID',how='left')
    f4 = come.groupby('stationID').agg({'twice_come':'sum'}).rename(columns={'twice_come':'twice_come_by_s'})
    final = final.merge(f4,on='stationID',how='left')
    final['twice_to_once_come_s'] = final['once_come_by_s']/final['twice_come_by_s']
    
    f5 = come.groupby('minute').agg({'once_come':'sum'}).rename(columns={'once_come':'once_come_by_m'})
    final = final.merge(f5,on='minute',how='left')
    f6 = come.groupby('minute').agg({'twice_come':'sum'}).rename(columns={'twice_come':'twice_come_by_m'})
    final = final.merge(f6,on='minute',how='left')
    final['twice_to_once_come_m'] = final['once_come_by_m']/final['twice_come_by_m']
    # 数据格式减少内存
    final['date'] = date
    final.fillna(0,inplace=True)
    for c in final.dtypes.index:
        if final.dtypes[c] == 'float64':
            final[c] = final[c].astype('float32')
    all_df = all_df.append(final)
    

In [5]:
all_df.to_pickle('features/progress4.pkl')